# Interacción

_"Un gráfico no se 'dibuja' de una sola vez; se 'construye' y 'reconstruye' hasta que revela todas las relaciones constituidas por la interacción de los datos. Las mejores operaciones gráficas son las que realiza el propio tomador de decisiones"_ &mdash; [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)

La visualización proporciona un poderoso medio para dar sentido a los datos. Sin embargo, una sola imagen suele dar respuesta, en el mejor de los casos, a un puñado de preguntas. A través de la _interacción_ podemos transformar imágenes estáticas en herramientas de exploración: resaltando puntos de interés, haciendo zoom para revelar patrones granulares más fino y enlazando a través de múltiples vistas para razonar sobre relaciones multidimensionales.

En el centro de la interacción está la noción de una _selección_: un medio para indicar al ordenador qué elementos o regiones nos interesan. Por ejemplo, podemos pasar el ratón por encima de un punto, hacer clic en múltiples marcas, o dibujar un cuadro delimitador alrededor de una región para resaltar subconjuntos de datos para un mayor escrutinio.

Junto con las codificaciones visuales y las transformaciones de datos, Altair proporciona una *abstracción de selección* (_selection_ abstraction) para la creación de interacciones. Estas selecciones abarcan tres aspectos:

1. Manejo de eventos de entrada para seleccionar puntos o regiones de interés, tales como pasar el ratón por encima, hacer clic, arrastrar, desplazar y tocar.
2. Generalizar a partir de la entrada para formar una regla de selección o [_predicado(predicate)_](https://es.wikipedia.org/wiki/Predicado_(lógica)) que determine si un registro de datos dado se encuentra o no dentro de la selección.
3. Usar el predicado de selección para configurar dinámicamente una visualización mediante el manejo de _codificaciones condicionales_, _transformaciones de filtro_ o _dominios de escala_.

Este *notebook* introduce selecciones interactivas y explora cómo utilizarlas para crear una variedad de técnicas de interacción, tales como consultas dinámicas, *panning &amp; zooming*, detalles a pedido y *brushing &amp; linking*.

_Este notebook es una traduccion y modificación del notebook **"Introduction to Vega-Lite / Altair"** , el cual es parte de [data visualization curriculum](https://github.com/uwdata/visualization-curriculum)._

_Todos los notebooks en español se encuentran en: [Altair en Simplificando Datos](https://github.com/SimplificandoDatos/Altair)_


In [1]:
import pandas as pd
import altair as alt

## Datos

Visualizaremos una variedad de conjuntos de datos de la colección [vega-datasets](https://github.com/vega/vega-datasets):

- Los datos `cars` de automóviles de los años 70 y principios de los 80,
- Los datos `movies`, sobre películas y que previamente utilizamos en el *notebook* [Transformación de datos](https://github.com/SimplificandoDatos/Altair/blob/master/03_transformacion_de_datos_altair.ipynb),
- Un conjunto de datos que contiene diez años de precios de acciones de [S&amp;P 500](https://es.wikipedia.org/wiki/S%26P_500) (`sp500`),
- Los datos `stocks` que contiene información de las acciones de empresas de tecnología, y
- Los datos `flights` con información sobre vuelos, incluyendo la hora de salida, la distancia y la demora en la llegada.

In [2]:
cars = 'https://vega.github.io/vega-datasets/data/cars.json'
movies = 'https://vega.github.io/vega-datasets/data/movies.json'
sp500 = 'https://vega.github.io/vega-datasets/data/sp500.csv'
stocks = 'https://vega.github.io/vega-datasets/data/stocks.csv'
flights = 'https://vega.github.io/vega-datasets/data/flights-5k.json'

## Introducción a las selecciones

Comencemos con una selección básica: simplemente haciendo *click* en un punto para resaltarlo. Usando los datos de `cars`, comenzaremos con un gráfico de dispersión de los caballos de fuerza (horsepower) por galón, con una codificación de colores para el número de cilindros del motor. 

Además, crearemos una instancia de selección llamando `alt.selection_single()`, indicando que queremos una selección definida sobre un único valor (_single value_). Por defecto, la selección usa un *click* para determinar el valor seleccionado. Para registrar una selección en una gráfica, debemos añadirla usando el método `.add_selection()`.

Una vez definimos nuestra selección, podemos usarla como un parámetro para una _codificación condicional (conditional encodings)_, la cual aplica una codificación diferente dependiendo si el valor del dato se encuentra afuera o adentro de nuestra selección. Por ejemplo, considera el código siguiente:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~

Esta definición de codificación establece que los datos contenidos dentro de la selección (`selection`) deberán ser coloreados de acuerdo con el campo `Cylinder`, mientras que los otros serán coloreados por defecto en gris (`grey`). Una selección vacía incluye a _todos_ los datos y por ello inicialemente todos los puntos estarán coloreados.

_Prueba haciendo *click* en diferentes puntos de la gráfica siguiente. ¿Qué sucede? (Haz *click* en el fondo de la gráfica para limpiar la selección y regresar a la condición inicial.)_

In [3]:
selection = alt.selection_single();
  
alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


¡Por supuesto, resaltando puntos individuales, uno a la vez, no es particularmente emocioante! Como veremos, sin embargo, las selecciones de valores únicos proveen un útil bloque de construcción (piensa en una pieza de lego) para interacciones más poderosas. Además, las selecciones de valores únicos son solamente una de los tres tipos selecciones provistos por Altair:

- `selection_single` - selecciona un único valor discreto, por defecto con un *click*.
- `selection_multi` - selecciona múltiples valores discretos. El primer valor es seleccionado con un *click* y los valores adicionales usando *shift-click*.
- `selection_interval` - selecciona un rango continuo de valores, iniciado por un *mouse drag*.

Comparemos cada uno de estos tipos de selección.  Para mantener nuestro códico ordenado, primero definiremos una función (`plot`) que genera una gráfica de dispersión similar a la anterior. Podemos pasar *selection* a la función `plot` para tenerla aplicada a la gráfica:

In [4]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Usemos nuestra función `plot` para crear tres variantes de la misma gráfica, una por método de selección.

La primera gráfica (`single`) es una réplica de nuestro ejemplo anterior. La segunda gráfica (`multi`) utiliza shift-click para incluir varios puntos en la selección. La tercera gráfica (`interval`) selecciona una región con un *mouse drag*. Una vez creada la región puedes ampliarla o disminuirla con el *scroll*, también te puedes desplazar con un *click* sostenido.

_¡Prueba interactuando con cada una de las siguientes gráficas!_

In [5]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Los ejemplos anteriores usan las interacciones por defecto (*click, shift-click, drag*) para cada tipo de selección. Podemos personalizar mucho más las interacciones proporcionando especificaciones de entrada usando [la sintaxis del selector de eventos de Vega (Vega event selector syntax)](https://vega.github.io/vega/docs/event-streams/). Por ejemplo, podemos modificar nuestras gráficas `single` y `multi` para que selecciones con un evento `mouseover` en lugar de un `click`

_¡Mantén presionada la tecla ***shift*** para "pintar" con datos!_

In [6]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Ahora que cubrimos lo basico de las selecciones con Altair, ¡Hagamos un recorrido por las diversas técnicas de interacción que permiten!

## Dynamic Queries (Consultas dinámicas)

_Dynamic queries_ permite una exploración de los datos rápida y reversible para aislar patrones de interés. En [Ahlberg, Williamson, &amp; Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), una consulta dinámica se define como aquella que:

- representa una consulta de manera gráfica,
- provee límites visibles del rango de la consulta,
- provee una representación gráfica de los datos y el resultado de la consulta,
- da una retroalimentación inmediata del resultado luego de cada modificación de la consulta,
- y permite a novates comenzar a trabajar con poca capacitación.

Un enfoque común es manipular los parámetros de la consulta usando *widgets* en la interfaz de usuaries como *sliders*, botones de radio y menús desplegables. Para generar *widgets* de consultas dinámica, podemos aplicar la operación `bind` de una selección a uno o más campos de los datos que deseamos consultar.

Construyamos una gráfica de dispersión que use una consulta dinámica para filtrar la visualización. Dada una gráfica de dispersión de la calificación de las películas (de Rotten Tomates e IMDB), podemos añadir una selección sobre el campo `Major_Genre` que nos permita filtrar interactivamente por el género de la película.

Para iniciar, extraigamos los géneros únicos (non-null) del conjunto de datos `movies`.

In [7]:
df = pd.read_json(movies) # cargar los datos
genres = df['Major_Genre'].unique() # obtener valores únicos
genres = list(filter(lambda d: d is not None, genres)) # filtrar los valores nulos
genres.sort() #  ordenar alfabéticamente

Para un uso posterior, definamos una lista de valores únicos de `MPAA_Rating`:

In [8]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Ahora creemos una selección única (`single`) unida a un menú desplegable.

*Utiliza el menú de consulta dinámica que aparece a continuación para explorar los datos. ¿Cómo varían las calificaciones por género? ¿Cómo revisarías el código para filtrar la `MPAA_Rating` (G, PG, PG-13, etc.) en lugar de `Major_Genre`?*

In [20]:
selectGenre = alt.selection_single(
    name='Selecciona', # nombre de la selección 'Select'
    fields=['Major_Genre'], # limita la selección a el campo Major_Genre
    init={'Major_Genre': genres[0]}, # usar el primer valor de género como valor inicial
    bind=alt.binding_select(options=genres) # relaciona el menú a valores únicos del género
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Nuestra construcción de arriba aprovecha múltiples aspectos de las selecciones:
- Le damos un nombre a la selección (`'Selecciona'`). Este nombre no es requerido, pero nos permite influir en el texto de la ética del menú de consulta dinámica que se generó. (_¿Qué pasa si remueves el nombre? ¡Haz la prueba!_)
- Delimitamos la selección a un campo específico (`Major_Genre`). Anteriormente, cuando usábamos una selección `single`, la selección se asignaba a puntos individuales. Limitando la selección a un campo específico, podemos seleccionar _todos_ los datos cuyo valor del campo `Major_Genre` coincide con el valor seleccionado en el menú.
- Inicializamos la selección a un valor inicial `init=...`.
- Vinculamos la selección a un *widget* de interfaz, en este caso a un menú desplegable por medio de `binding_select`.
- Al igual que antes, utilizamos una codificación condicional para controlar la opacidad del canal.

### Vinculando selecciones a múltiples entradas

Una instancia de selección puede vincularse a _múltiples_ *widgets* de consultas dinámicas. Modifiquemos el ejemplo anterior para proveer filtros para _ambos_ campos, `Major_Genre` and `MPAA_Rating`, usando botones de radio en lugar de un menú. Nuestra selección `single` ahora se define sobre un _par_ de valores de género y clasificación MPAA.

_Busca las sorprendentes conjunciones de género y clasificación. ¿Hay películas de terror con clasificación G o PG?_

In [10]:
# selección de valores individuales sobre pares [Major_Genre, MPAA_Rating]
# utilizar valores específicos como los valores iniciales seleccionados
selection = alt.selection_single(
    name='Selecciona',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)
  
# gráfico de dispersión, modifica la opacidad basado en la selección
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Dato curioso: La clasificación PG-13 no existía cuando las películas [Jaws (Tiburón)](https://www.imdb.com/title/tt0073195/) y [Jaws 2](https://www.imdb.com/title/tt0077766/) fueron lanzadas. La primera película en recibir la clasificación PG-13 fue en 1984 [Red Dawn](https://www.imdb.com/title/tt0087985/)._

### Usando visualizaciones como consultas dinámicas

Aunque los widgets de interfaz estándar muestran los valores de los parámetros de consulta _posible_, no visualizan la _distribución_ de esos valores. También podríamos querer usar interacciones más ricas, como selecciones multivalores o de intervalos, en lugar de widgets de entrada que seleccionan solo un valor a la vez.

Para resolver estos problemas, podemos crear gráficos adicionales tanto para visualizar los datos como para soportar consultas dinámicas. Añadamos un histograma del recuento de películas por año y utilicemos una selección de intervalos para resaltar dinámicamente las películas en los períodos de tiempo seleccionados.

*Interactúa con el histograma de los años para explorar películas de diferentes períodos de tiempo. ¿Haz visto alguna evidencia de [sesgo muestral](https://es.wikipedia.org/wiki/Sesgo_muestral) a través de los años? (¿Cómo se relacionan el año y las valoraciones de los críticos?)*

_¡Los años van de 1930 a 2040! ¿Están las futuras películas en pre-producción, o hay errores "off-by-one century"? Además, dependiendo de la zona horaria en la que te encuentres, es posible que veas un pequeño bache en 1969 o 1970. ¿Por qué puede ser eso? (¡Ve al final del cuaderno para una explicación!)_

In [11]:
brush = alt.selection_interval(
    encodings=['x'] # limita la selección a los valores del eje (year)
)

# histograma de consulta dinámica
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Películas por año de estreno'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# gráfico de dispersión, modifica la opacidad basado en la selección
ratings = alt.Chart(movies).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', title='Calificación por Rotten_Tomatoes'),
    alt.Y('IMDB_Rating:Q', title='Calificación por IMDB'),
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


El ejemplo anterior provee consultas dinámicas usando una selección enlazada entre gráficas:

- Creamos una selección de `intervalo` (`brush`), y ponemos `encodings=['x']` para limitar la selección al eje x solamente, resultando en un intervalo de selección unidimensional.
- Registramos `brush` en nuestro histograma de películas por año a través de `.add_selection(brush)`.
- Usamos `brush` en una codificación condicional para ajustar la `opacidad` de la gráfica de dispersión.

Esta técnica de interacción de seleccionar elementos en un gráfico y ver los resaltados enlazados en uno o más gráficos se conoce como [_brushing &amp; linking_](https://en.wikipedia.org/wiki/Brushing_and_linking).


## Panning &amp; Zooming (Enfoque/Paneo y zoom)

La gráfica de dispersión de la calificación de las películas está un poco desordenada en algunas partes, lo que dificulta el examen de los puntos en regiones más densas. Usando las técnicas de interacción de _panning_ y _zooming_, podemos inspeccionar regiones densas más de cerca.

Comencemos pensando en cómo podríamos expresar el paneo y el zoom usando las selecciones de Altair. ¿Qué define la "vista" de un gráfico? _¡Los dominios de la escala de los ejes!_

Podemos cambiar los dominios de escala para modificar el rango visualizado de los valores de los datos. Para hacerlo de forma interactiva, vinculamos una selección de `interval` a los dominios de escala con el código `bind='scales'`. El resultado es que en lugar de un *brush* de intervalos que podemos arrastrar y acercar, ¡Arrastramos y acercamos toda el área de la gráfica!

En el siguiente gráfico, haz *click* y arrastra para desplazar la vista, o haz *scroll* para ampliar la vista. ¿Qué puedes descubrir sobre la precisión de los valores de calificación proporcionados?

In [12]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x=alt.X('Rotten_Tomatoes_Rating:Q', title = 'Calificación Rotten_Tomatoes'),
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30), title = 'Calificación IMDB'), # usar la extensión mínima para estabilizar la colocación del título del eje
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_¡Con el *zoom*, podemos ver que los valores de calificación tienen una precisión limitada! Las calificaciones de Rotten Tomatoes son enteros, mientras que las calificaciones de IMDB están truncadas a décimas. Como resultado, hay un sobresaturación incluso cuando hacemos zoom, con múltiples películas que comparten los mismos valores de calificación._

Al leer el código anterior, puedes notar el código `alt.Axis(minExtent=30)` en el canal de codificación `y`. El parámetro `minExtent` asegura que se reserve una cantidad mínima de espacio para los ticks y etiquetas de los ejes. ¿Por qué hacer esto? Cuando nos desplazamos y hacemos zoom, las etiquetas de los ejes pueden cambiar y provocar que la posición del título del eje se desplace. Al establecer un mínimo podemos reducir los movimientos de distracción en el gráfico. _Intenta cambiar el valor de `minExtent`, por ejemplo, poniéndolo a cero, y luego aleja el zoom para ver qué sucede cuando entran en la vista etiquetas de ejes más largas._

Altair también incluye una forma abreviada para agregar el desplazamiento y el zoom a un gráfico. En lugar de crear directamente una selección, puede llamar a `.interactive()` para que Altair genere automáticamente una selección de intervalo ligada a las escalas del gráfico:

In [13]:
alt.Chart(movies).mark_circle().encode(
    x=alt.X('Rotten_Tomatoes_Rating:Q', title = 'Calificación Rotten_Tomatoes'),
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30), title = 'Calificación IMDB'), # usar la minExtent para estabilizar la colocación del título del eje
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Por defecto, los vínculos de escala para las selecciones incluyen los canales de codificación `x` y `y` . ¿Qué pasa si queremos limitar el paneo y el zoom a lo largo de una sola dimensión? Podemos invocar `encodings=['x']` para restringir la selección al canal `x` solamente:

In [14]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x=alt.X('Rotten_Tomatoes_Rating:Q', title = 'Calificación Rotten_Tomatoes'),
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30), title = 'Calificación IMDB'),  #usar la minExtent para estabilizar la colocación del título del eje
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Cuando se hace un zoom a lo largo de un solo eje, la forma de los datos visualizados puede cambiar, afectando potencialmente nuestra percepción de las relaciones en los datos. ¡[Elegir una relación de aspecto apropiada](http://vis.stanford.edu/papers/arclength-banking) es una preocupación importante en el diseño de la visualización!_

## Navegación: visión general + detalle

Cuando se hace paneo y zoom, ajustamos directamente el "viewport" de una gráfica. La estrategia de navegación relacionada de _overview + detail (visión general + detalle)_ utiliza en su lugar una vista general para mostrar _todos_ los datos, mientras que apoya las selecciones por paneo y zoom en una pantalla de enfoque separada.

A continuación tenemos dos gráficas de área que muestran una década de fluctuaciones de precios para el índice de acciones S&amp;P 500. Inicialmente ambas muestran el mismo rango de datos. _Haga clic y arrastre el gráfico de la parte inferior para actualizar la pantalla de enfoque y examinar períodos de tiempo específicos._

In [15]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q', title='Precio')
).properties(
    width=700
)
  
alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


A diferencia de nuestro caso anterior de panning &amp; zooming, aquí no queremos vincular una selección directamente a las escalas de un único gráfico interactivo. En su lugar, queremos vincular la selección a un dominio de escala en _otro_ gráfico. Para ello, actualizamos el canal de codificación `x` para nuestra gráfica de foco (la superior), estableciendo la propiedad de escala `domain` para referenciar nuestra selección `brush`. Si no se define ningún intervalo (la selección está vacía), Altair ignora el `brush` y utiliza los datos subyacentes para determinar el dominio. Cuando se crea un *brush interval*, Altair lo usa como el `domain` de escala para la gráfica de foco.

## Detalles a petición (Details on Demand)

Una vez que localizamos puntos de interés dentro de una visualización, a menudo queremos saber más sobre ellos. La expresión _detalles bajo demanda/detalles a petición_ se refiere a la consulta interactiva para obtener más información sobre los valores seleccionados. Los _Tooltips_ son un medio útil para proporcionar detalles bajo demanda. Sin embargo, los *tooltips* normalmente sólo muestran información para un punto de los datos a la vez. ¿Cómo podríamos mostrar más?

La gráfica de dispersión de la calificación de las películas incluye una serie de valores atípicos potencialmente interesantes en los que las calificaciones de Rotten Tomatoes e IMDB no coinciden. Vamos a crear una gráfica que nos permita seleccionar puntos de forma interactiva y mostrar sus etiquetas.

_Pasa el ratón por encima de los puntos de la gráfica de dispersión que aparece a continuación para ver un resaltado y una etiqueta del título. Haz **Shift+click**  en los puntos para que las anotaciones sean persistentes y veas varias etiquetas a la vez. ¿Qué películas les gustan a los críticos de Rotten Tomatoes, pero no al público en general en IMDB (o viceversa)? Ve si puedes encontrar posibles errores, donde dos películas diferentes con el mismo nombre se combinaron accidentalmente!_

In [16]:
hover = alt.selection_single(
    on='mouseover',  # seleccionar con el paso del mouse
    nearest=True,    # seleccionar el punto más cercano al cursor del ratón
    empty='none'     # sin selección no se relaciona ningún punto
)

click = alt.selection_multi(
    empty='none' # sin selección no se relaciona ningún punto
)

# la codificación de la gráfica de dispersión es compartida por todos los marcadores
plot = alt.Chart().mark_circle().encode(
    x=alt.X('Rotten_Tomatoes_Rating:Q', title = 'Calificación Rotten_Tomatoes'),
    y=alt.Y('IMDB_Rating:Q',title = 'Calificación IMDB')
)
  
# el gráfico base compartido para nuevas capas
base = plot.transform_filter(
    # logica "OR/" es suportado por Vega-Lite
    {'or': [hover, click]} # filtra a los puntos en la selección
)

alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


El ejemplo anterior añade tres nuevas capas a la gráfica de dispersión: una anotación circular, texto blanco para proporcionar un fondo legible y texto negro que muestra el título de una película. Además, este ejemplo utiliza dos selecciones en combinación:

1. Una sola selección (`hover`) que incluye `nearest=True` para seleccionar automáticamente el punto de datos más cercano a medida que el ratón se mueve.
2. Una selección múltiple (`click`) para crear selecciones persistentes mediante shift+click.

Ambas selecciones incluyen el conjunto `empty='none'` para indicar que no se deben incluir puntos si una selección está vacía. Estas selecciones se combinan entonces en un único filtro predicado &mdash; el lógico _or_ de `hover` y `click` &mdash; para incluir puntos que residen en _either_ selection. Usamos este predicado para filtrar las nuevas capas para mostrar anotaciones y etiquetas sólo para los puntos seleccionados.

¡Utilizando selecciones y capas, podemos realizar un número de diferentes diseños para los detalles a petición! Por ejemplo, aquí se muestra una serie temporal a escala logarítmica de los precios de las acciones de tecnología, anotada con una línea y etiquetas para la fecha más cercana al cursor del mouse:

In [17]:
# seleccionar un punto para los cuales proveer detalles bajo demanda
label = alt.selection_single(
    encodings=['x'], # limitar la selección a los valores del eje X
    on='mouseover',  # seleccionar con le paso del mouse
    nearest=True,    # seleccionar los puntos de datos máas cercanos al cursor
    empty='none'     # sin selección no se relaciona ningún punto
)

# definir nuestra gráfica de línea base para los precios de los "stock"
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfica base
    
    # añadir un marcador de regla para servir como línea guía
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),
    
    # añadir marcas de círculos para los puntos de tiempo seleccionados, ocultar puntos no seleccionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # añadir texto en trazos blancos para proporcionar un fondo legible para las etiquetas
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # añadir etiquetas de texto para el precio de los "stock"
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),
    
    data=stocks
).properties(
    width=700,
    height=400
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Poniendo en acción lo que hemos aprendido hasta ahora: ¿puedes modificar la gráfica de dispersión de la película de arriba (la que tiene la consulta dinámica sobre años) para incluir una marca de "regla" que muestre la calificación promedio de IMDB (o Rotten Tomatoes) para los datos contenidos en la selección de "intervalo" de años?_

## Brushing &amp; Linking, Revisado

Anteriormente en este *notebook* vimos un ejemplo de _brushing &amp; linking_: uso de un histograma de consulta dinámica para resaltar puntos en una gráfica de dispersión de calificación de películas. Aquí veremos algunos ejemplos adicionales que involucran selecciones enlazadas.

Volviendo al conjunto de datos `cars`, podemos usar el operador `repeat` para construir una matriz [scatter plot matrix (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que muestra asociaciones entre kilometraje, aceleración y caballos de fuerza. Podemos definir una selección `interval` e incluirla _dentro_ de nuestra especificación de dispersión repetida para permitir selecciones vinculadas entre todas las gráficas.

_¡Haga clic y arrastre en cualquiera de las gráficas de abajo para realizar brushing &amp; linking!_

In [22]:
brush = alt.selection_interval(
    resolve='global' # resolver todas las selecciones a una sola instancia global
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Observa arriba el uso de `resolve='global'` en la selección `interval`. El ajuste por defecto `'global'` indica que en todas las gráficas sólo puede estar activa una *brush* a la vez. Sin embargo, en algunos casos podríamos querer definir *brushes* en múltiples gráficos y combinar los resultados. Si utilizamos `resolve='union'`, la selección será la _unión_ de todas las brochas/brushes: si un punto reside dentro de cualquier brocha/brush será seleccionado. Alternativamente, si usamos `resolve='intersect'`, la selección consistirá en la _intersección_ de todas las brochas/brushes: sólo se seleccionarán los puntos que residan dentro de todas las escobillas.

_Intenta poner el parámetro `resolve` a `'union'` e `'intersect'` y ve cómo cambia la lógica de selección resultante._

### Cross-Filtering (Filtros cruzados)

Los ejemplos de brushing &amp; linking que hemos visto usan codificaciones condicionales, por ejemplo para cambiar los valores de opacidad en respuesta a una selección. Otra opción es utilizar una selección definida en una vista para _filtrar_ el contenido de otra vista.

Construyamos una colección de histogramas para el conjunto de datos `flights`. Usaremos el operador `repeat` para crear los histogramas, y añadiremos una selección de `interval` para el eje `x` con las brochas/brushes resueltas por intersección.

En particular, cada histograma consistirá en dos capas: una capa de fondo gris y una capa de primer plano azul, con la capa de primer plano filtrada por nuestra intersección de *brush selections*. ¡El resultado es una interacción de _filtrado cruzado_ a través de los tres gráficos!

_Arrastre los intervalos de brush en los gráficos de abajo. Al seleccionar vuelos con retrasos de llegada más largos o más cortos, ¿cómo responden las distribuciones de distancia y tiempo?_

In [19]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # hasta  100 bins
        axis=alt.Axis(format='d', titleAnchor='start')
    ),
    alt.Y('count():Q', title=None)
)
  
alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # retrasos > 3 horas
    time='hours(datum.date) + minutes(datum.date) / 60' # en fracciones de horas
).configure_view(
    stroke='transparent'
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Por medio del filtro cruzado puedes observar que los vuelos retrasados tienen más probabilidades de salir a horas más tardías. Este fenómeno es familiar para los viajeros frecuentes: un retraso puede propagarse a lo largo del día, afectando al viaje posterior de ese avión. ¡Para tener las mejores probabilidades de una llegada a tiempo, reserve un vuelo temprano!_

¡La combinación de múltiples vistas y selecciones interactivas puede permitir valiosas formas de razonamiento multidimensional, convirtiendo incluso los histogramas básicos en poderosos dispositivos de entrada para hacer preguntas de un conjunto de datos!

## Resumen

Para más información sobre las opciones de interacción soportadas en Altair, por favor consulta la [documentación de selección interactiva de Altair](https://altair-viz.github.io/user_guide/interactions.html). Para detalles sobre la personalización de los manejadores de eventos, por ejemplo para componer técnicas de interacción múltiple o para soportar entrada táctil en dispositivos móviles, consulte la [documentación de selección de Vega-Lite](https://vega.github.io/vega-lite/docs/selection.html).

¿Estás interesado en saber más?
- La abstracción de la _selección_ fue introducida en el artículo [Vega-Lite: Una Gramática de Gráficos Interactivos](http://idl.cs.washington.edu/papers/vega-lite/), por Satyanarayan, Moritz, Wongsuphasawat, &amp; Heer.
- El sistema PRIM-9 (para proyección, rotación, aislamiento y enmascaramiento en hasta 9 dimensiones) es una de las primeras herramientas de visualización interactiva, construida a principios de los años 70 por Fisherkeller, Tukey, &amp; Friedman. [¡Un vídeo de demostración retro sobrevive!](http://stat-graphics.org/movies/prim9.html)
- El concepto de  brushing &amp; linking fue cristalizado por Becker, Cleveland, &amp; Wilks en su artículo de 1987 [Gráficos dinámicos para el análisis de datos](https://scholar.google.com/scholar?cluster=14817303117298653693).
- Para un resumen completo de las técnicas de interacción para la visualización, ver [Interactive Dynamics for Visual Analysis](https://queue.acm.org/detail.cfm?id=2146416) de Heer &amp; Shneiderman.
- Finalmente, para un tratado sobre lo que hace que la interacción sea efectiva, lee el clásico [Direct Manipulation Interfaces](https://scholar.google.com/scholar?cluster=15702972136892195211) de Hutchins, Hollan, &amp; Norman.

#### Anexo: sobre la representación del tiempo

Anteriormente observamos un pequeño incremento en el número de películas en 1969 y 1970. ¿De dónde viene ese salto? ¿Y por qué 1969 o 1970? La respuesta proviene de una combinación de datos que faltan y de cómo tu computadora representa el tiempo. 

Internamente, las fechas y las horas se representan en relación con el [Tiempo UNIX](https://es.wikipedia.org/wiki/Tiempo_Unix), en la que el tiempo "cero" corresponde al golpe de medianoche del 1 de enero de 1970 en la [hora UTC](https://es.wikipedia.org/wiki/Tiempo_universal_coordinado), que corre a lo largo del [meridiano principal](https://es.wikipedia.org/wiki/Meridiano_cero). Resulta que hay algunas películas a las que les faltan ( `null`) fechas de estreno. Esos valores `null` se interpretan como la hora `0`, y por lo tanto se asignan al 1 de enero de 1970 en tiempo UTC. Si vives en las Américas y en las zonas horarias "más tempranas", este punto exacto en el tiempo corresponde a una hora más temprana del 31 de diciembre de 1969 en tu zona horaria local. Por otro lado, si vives cerca o al este del meridiano principal, la fecha en tu zona horaria local será el 1 de enero de 1970.

¿El consejo? ¡Siempre debes ser escéptico con respecto a tus datos, y ten en cuenta que la forma en que se representan los datos (ya sea como fechas y horas, o números de punto flotante, o latitudes y longitudes, _etc._) a veces puede conducir a artefactos que impactan en el análisis!